In [1]:
!docker volume create psql

psql


In [2]:
!docker volume ls

DRIVER    VOLUME NAME
local     psql


In [4]:
!docker volume inspect psql

[
    {
        "CreatedAt": "2025-08-10T22:24:21Z",
        "Driver": "local",
        "Labels": null,
        "Mountpoint": "/var/lib/docker/volumes/psql/_data",
        "Name": "psql",
        "Options": null,
        "Scope": "local"
    }
]


In [4]:
!docker container stop psql1

psql1


In [5]:
!docker run -d --name psql1 -e POSTGRES_PASSWORD=mypassword -p 5433:5432 -v psql:/var/lib/postgresql/data postgres:15.1

ce8ade4283f2370e99af0c2e40e0fc561b4dfd80b69252c397170963621274da


In [ ]:
!docker container ls

CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS          PORTS                                     NAMES
edb923330129   postgres:15.1   "docker-entrypoint.s…"   10 seconds ago   Up 9 seconds    5432/tcp                                  psql1
2367b5fdd97a   nginx           "/docker-entrypoint.…"   13 minutes ago   Up 13 minutes   0.0.0.0:8080->80/tcp, [::]:8080->80/tcp   nginx2
6b099ff6ca43   nginx           "/docker-entrypoint.…"   14 minutes ago   Up 14 minutes   0.0.0.0:80->80/tcp, [::]:80->80/tcp       nginx


In [7]:
import psycopg2

# Connection settings (adjust if needed)
conn = psycopg2.connect(
    host="localhost",       # or "psql1" if running from another container
    port=5433,
    user="postgres",
    password="mypassword",
    dbname="postgres"
)

cur = conn.cursor()

# Create table
cur.execute("""
    CREATE TABLE IF NOT EXISTS test_table (
        id SERIAL PRIMARY KEY,
        name TEXT
    )
""")

# Insert 1 record
cur.execute("INSERT INTO test_table (name) VALUES (%s) RETURNING id", ("Hello from Python",))
new_id = cur.fetchone()[0]
print(f"Inserted row with ID: {new_id}")

# Commit the transaction
conn.commit()

# Select and print all rows
cur.execute("SELECT id, name FROM test_table")
for row in cur.fetchall():
    print(row)

# Cleanup
cur.close()
conn.close()


Inserted row with ID: 1
(1, 'Hello from Python')


In [8]:
!docker logs psql1


PostgreSQL Database directory appears to contain a database; Skipping initialization



2025-08-10 22:34:50.464 UTC [1] LOG:  starting PostgreSQL 15.1 (Debian 15.1-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit
2025-08-10 22:34:50.465 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2025-08-10 22:34:50.465 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2025-08-10 22:34:50.484 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2025-08-10 22:34:50.506 UTC [29] LOG:  database system was shut down at 2025-08-10 22:34:34 UTC
2025-08-10 22:34:50.519 UTC [1] LOG:  database system is ready to accept connections


In [9]:
!docker stop psql1

psql1


In [12]:
!docker run -d --name psql2 -e POSTGRES_PASSWORD=mypassword -p 5433:5432 -v psql:/var/lib/postgresql/data postgres:15.2

f03c9c4822e2d8a71d6646e80a08516aa7edd7105a173419eade56dfefc56bf2


Unable to find image 'postgres:15.2' locally
15.2: Pulling from library/postgres
3ef71fe7cece: Pulling fs layer
7782b3e1be4b: Pulling fs layer
247ec4ff783a: Pulling fs layer
8cf43a35dc10: Pulling fs layer
b3f90da87261: Pulling fs layer
f7ead6900700: Pulling fs layer
5f7e3ef14a3b: Pulling fs layer
dd8e229a2a13: Pulling fs layer
e7afdbe9a191: Pulling fs layer
9e3ea8720c6d: Pulling fs layer
1459ebb56be5: Pulling fs layer
3595124f6861: Pulling fs layer
2fac9b4aab2f: Pulling fs layer
7782b3e1be4b: Download complete
b3f90da87261: Download complete
5f7e3ef14a3b: Download complete
8cf43a35dc10: Download complete
1459ebb56be5: Download complete
2fac9b4aab2f: Download complete
3595124f6861: Download complete
247ec4ff783a: Download complete
f7ead6900700: Download complete
3ef71fe7cece: Download complete
e7afdbe9a191: Download complete
dd8e229a2a13: Download complete
9e3ea8720c6d: Download complete
9e3ea8720c6d: Pull complete
7782b3e1be4b: Pull complete
247ec4ff783a: Pull complete
f7ead6900700: Pu

In [13]:
import psycopg2

# Connection settings (adjust if needed)
conn = psycopg2.connect(
    host="localhost",       # or "psql1" if running from another container
    port=5433,
    user="postgres",
    password="mypassword",
    dbname="postgres"
)

cur = conn.cursor()


# Select and print all rows
cur.execute("SELECT id, name FROM test_table")
for row in cur.fetchall():
    print(row)

# Cleanup
cur.close()
conn.close()


(1, 'Hello from Python')
